In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm

In [2]:
RR = 0.1
r = 0

 # Q1.

In [3]:
kappa = 4
long_term_lambda = 0.01
beta = 0.05
lambda_1 = 0.01
simul_num = 100000
N_member = 25
Triger = 5
dt = 1/52
years = 5

In [4]:
lambda_lst = np.zeros((52*years + 1,simul_num)).astype(np.float32)
weekly_default = np.zeros((52*years + 1,simul_num)).astype(np.float32)

lambda_lst[0] = lambda_1
for i in range(1,52*years+1) :
    num_of_default = (np.random.uniform(0,1,(N_member,simul_num ))< lambda_lst[i-1] * dt).sum(0)
    dlambda = kappa*(long_term_lambda - lambda_lst[i-1]) * dt + beta * num_of_default
    new_lambda = dlambda + lambda_lst[i-1]
    weekly_default[i] = num_of_default
    lambda_lst[i] = np.minimum(new_lambda ,1)
    
ary = np.zeros((12*years, simul_num))
month1_weeks = np.array(([4]*8 + [5]*4)*years)
b = month1_weeks.cumsum()+1
a = month1_weeks.cumsum()-month1_weeks +1
for i in range(len(month1_weeks)) :
    ary[i] = weekly_default[a[i]:b[i]].sum(0)    

In [5]:
((ary>=5).sum(0)>0).sum()

6

 # Functionize

In [6]:
def defaulted_Case(kappa = 4, long_term_lambda = 0.01, beta = 0.05,
                   lambda_1 = 0.01, simul_num = 10**5, N_member = 25, Triger = 5, dt = 1/52, years = 5) :
    lambda_lst = np.zeros((52*years + 1,simul_num)).astype(np.float32)
    weekly_default = np.zeros((52*years + 1,simul_num)).astype(np.float32)
    
    lambda_lst[0] = lambda_1
    for i in range(1,52*years+1) :
        num_of_default = (np.random.uniform(0,1,(N_member,simul_num ))< lambda_lst[i-1] * dt).sum(0)
        dlambda = kappa*(long_term_lambda - lambda_lst[i-1]) * dt + beta * num_of_default
        new_lambda = dlambda + lambda_lst[i-1]
        weekly_default[i] = num_of_default
        lambda_lst[i] = np.minimum(new_lambda ,1)
    
    ary = np.zeros((12*years, simul_num))
    month1_weeks = np.array(([4]*8 + [5]*4)*years)
    b = month1_weeks.cumsum()+1
    a = month1_weeks.cumsum()-month1_weeks +1
    for i in range(len(month1_weeks)) :
        ary[i] = weekly_default[a[i]:b[i]].sum(0)  
    
    N_of_Default = ((ary>=5).sum(0)>0).sum()    
    return N_of_Default
    
def defaulted_Case2(kappa = 4, long_term_lambda = 0.01, beta = 0.05,
                    lambda_1 = 0.01, simul_num = 100000, N_member = 25, Triger = 5, dt = 1/52, years = 5) :
    lambda_lst = np.zeros((52*years + 1,simul_num)).astype(np.float32)
    weekly_default = np.zeros((52*years + 1,simul_num)).astype(np.float32)
    
    lambda_lst[0] = lambda_1
    for i in range(1,52*years+1) :
        num_of_default = (np.random.uniform(0,1,(N_member,simul_num ))< lambda_lst[i-1] * dt).sum(0)
        dlambda = kappa*(long_term_lambda - lambda_lst[i-1]) * dt + beta * num_of_default
        new_lambda = dlambda + lambda_lst[i-1]
        weekly_default[i] = num_of_default
        lambda_lst[i] = np.minimum(new_lambda ,1)
    
    ary = np.zeros((12*years, simul_num))
    month1_weeks = np.array(([4]*8 + [5]*4)*years)
    b = month1_weeks.cumsum()+1
    a = month1_weeks.cumsum()-month1_weeks +1
    for i in range(len(month1_weeks)) :
        ary[i] = weekly_default[a[i]:b[i]].sum(0)  
        
    return ((ary>=5).sum(0)>0)
        

In [7]:
defaulted_Case()

6

In [8]:
defaulted_Case(beta = 0.15)

5209

In [9]:
def CVA(kappa = 4, long_term_lambda = 0.01, beta = 0.05,        
        lambda_1 = 0.01, simul_num = 100000, N_member = 25, Triger = 5,
        dt = 1/52, years = 5 , initial_F = 1000000, RR = 0.1, sigma = 0.2) :

    Def_Case = defaulted_Case2(kappa = kappa, long_term_lambda = long_term_lambda, beta = beta,
                               lambda_1 = lambda_1, simul_num = simul_num, N_member = N_member,
                               Triger = Triger, dt = dt, years = years)
    
    F = initial_F
    epsilon = np.random.normal(0,1,(52 * years,simul_num))
    F_path = F * np.exp((sigma*np.sqrt(dt) * epsilon).cumsum(0))
    Exposure = (F_path[-1,:]-F) * (F_path[-1,:]>F)
    CVA = (Def_Case * Exposure * (1-RR)).sum()/simul_num
    return CVA

In [10]:
#Default Case to 1year
defaulted_Case(kappa = 2, long_term_lambda=0.02, lambda_1 = 0.02,simul_num=100000, beta = 0.1, years = 1)

330

In [11]:
#CVA 1year
CVA(kappa = 2, long_term_lambda=0.02, lambda_1 = 0.02,simul_num=100000, beta = 0.1, years = 1, sigma = 0.2,initial_F = 1000000)

263.6643154772026

 # 2-(b)

In [13]:
def defaulted_Case3(kappa = 4, long_term_lambda = 0.01, beta = 0.05,
                    lambda_1 = 0.01, simul_num = 100000, N_member = 25, Triger = 5, dt = 1/52, years = 5) :
    lambda_lst = np.zeros((52*years + 1,simul_num)).astype(np.float32)
    weekly_default = np.zeros((52*years + 1,simul_num)).astype(np.float32)
    
    lambda_lst[0] = lambda_1
    my_CDS_reference_number = 1
    my_CDS_reference_default= np.zeros((52*years+1,simul_num)) 
    for i in range(1,52*years+1) :
        firm_default = (np.random.uniform(0,1,(N_member,simul_num ))< lambda_lst[i-1] * dt)
        my_CDS_reference_default[i] = firm_default[my_CDS_reference_number,:]
        num_of_default = firm_default.sum(0)
        dlambda = kappa*(long_term_lambda - lambda_lst[i-1]) * dt + beta * num_of_default
        new_lambda = dlambda + lambda_lst[i-1]
        weekly_default[i] = num_of_default
        lambda_lst[i] = np.minimum(new_lambda ,1)
    
    ary = np.zeros((12*years, simul_num))
    month1_weeks = np.array(([4]*8 + [5]*4)*years)
    b = month1_weeks.cumsum()+1
    a = month1_weeks.cumsum()-month1_weeks +1
    for i in range(len(month1_weeks)) :
        ary[i] = weekly_default[a[i]:b[i]].sum(0)
        
    return ((ary>=5).sum(0)>0) , (my_CDS_reference_default.sum(0)>0)

In [14]:
CCP_DF, CDS_reference_DF = defaulted_Case3(kappa = 2, long_term_lambda=0.02, lambda_1 = 0.02,simul_num=100000, beta = 0.1, years = 5)

In [15]:
RR_reference = 0.5
Principal = 1000000
Exposure = CDS_reference_DF * ( 1-RR_reference)  * Principal

In [16]:
RR = 0.1
simul_num = 100000
CVA_CDS = (Exposure * CCP_DF * (1-RR)).sum()/simul_num

In [17]:
CVA_CDS

99076.5

 # Q3.

In [24]:
def CVA_bond(kappa = 4, long_term_lambda = 0.01, beta = 0.05,        
             lambda_1 = 0.01, simul_num = 100000, N_member = 25, Triger = 5,
             dt = 1/52, years = 5 , initial_F = 1000000, RR = 0.1, sigma = 0.2) :

    Def_Case = defaulted_Case2(kappa = kappa, long_term_lambda = long_term_lambda, beta = beta,
                               lambda_1 = lambda_1, simul_num = simul_num, N_member = N_member,
                               Triger = Triger, dt = dt, years = years)
    
    F = initial_F
    epsilon = np.random.normal(0,1,(52 * years,simul_num))
    CVA = (Def_Case * F * (1-RR)).sum()/simul_num
    return CVA

In [25]:
CVA_b = CVA_bond(kappa = 2, long_term_lambda=0.02, lambda_1 = 0.02,simul_num=100000, beta = 0.1, years = 5, sigma = 0.2,initial_F = 1000000)
CVA_b

220194.0

In [26]:
CVA_b/(1000000)

0.220194

In [27]:
DP_CCP = defaulted_Case(kappa = 2, long_term_lambda=0.02, lambda_1 = 0.02,simul_num=100000, beta = 0.1, years = 5)
DP_CCP/(10**5)

0.24326

In [28]:
DP_CCP/(10**5) * (1-RR)

0.21893400000000002